# PyTorch with The Fashion MINST dataset

This notebook contains the steps for building deep neural network using PyTorch where it takes an image of clothes as input and predict the class of that image.

## 1. Import The Relevant Packages

In [10]:
import torch 
import torch.nn as nn 
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
device = 'cuda' if torch.cuda.is_available() else 'cpu'


In [11]:
from torchvision import datasets
data_folder = "~/cv/pytorch_dataset/FMNIST"
fmnist = datasets.FashionMNIST(data_folder, download=True, train=True)

In [12]:
train_images = fmnist.data 
train_targets = fmnist.targets

In [ ]:
class FMNISTDataset(Dataset):
    def __init__(self,x,y):
        x= x.float()
        x = x.view(-1,28,28)
        self.x , self.y = x,y
    def __len__(self):
        return len(self.x)
    def __getitem__(self,index):
        x,y = self.x[index], self.y[index]
        return x.to(device), y.to(device)